In [1]:
import zipfile
from zipfile import ZipFile
import pandas as pd
import numpy as np
import netCDF4 as nc
from matplotlib import pyplot as plt
import xarray as xr
import os
from calendar import isleap
import warnings
warnings.filterwarnings("ignore")
import copy
import xarray as xr
from rasterio import Affine
from rasterstats import zonal_stats
from osgeo import gdal
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon

ModuleNotFoundError: No module named 'osgeo'

In [2]:
from shapely.ops import unary_union
gdf = gpd.read_file(r"D:\Mosquito Modeling\GTA boundaries\GTA.shp")
merged = gdf.geometry.unary_union
gdf_merged = gpd.GeoDataFrame(geometry=[merged])
gdf_merged = gdf_merged.set_crs('EPSG:4326')
# gdf_m = gdf_merged.to_crs('EPSG:3857')
# gdf_m['area_m2'] = gdf_m.geometry.area
# areaofshp=gdf_m['area_m2']

In [3]:
path="D:\Mosq Proj Data\CMIP Data\surface temperature\ssp585 NorESM2-MM r1i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\surface temperature\ssp585 NorESM2-MM r1i1p1f1"
average = {}
filepathname=[]
count=[]
for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    print(ds)

<xarray.Dataset>
Dimensions:    (time: 2190, bnds: 2, lat: 192, lon: 288)
Coordinates:
  * time       (time) object 2015-01-01 12:00:00 ... 2020-12-31 12:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tslsi      (time, lat, lon) float32 ...
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               ScenarioMIP
    branch_method:             Hybrid-restart from year 2015-01-01 of historical
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     60225.0
    ...                        ...
    title:                     NorESM2-MM output prepared for CMIP6
    variable_id:               tslsi
    variant_label:            

<xarray.Dataset>
Dimensions:    (time: 3650, bnds: 2, lat: 192, lon: 288)
Coordinates:
  * time       (time) object 2081-01-01 12:00:00 ... 2090-12-31 12:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tslsi      (time, lat, lon) float32 ...
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               ScenarioMIP
    branch_method:             Hybrid-restart from year 2015-01-01 of historical
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     60225.0
    ...                        ...
    title:                     NorESM2-MM output prepared for CMIP6
    variable_id:               tslsi
    variant_label:            

In [4]:
ds


<xarray.Dataset>
Dimensions:    (time: 3650, bnds: 2, lat: 192, lon: 288)
Coordinates:
  * time       (time) object 2091-01-01 12:00:00 ... 2100-12-31 12:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tslsi      (time, lat, lon) float32 ...
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               ScenarioMIP
    branch_method:             Hybrid-restart from year 2015-01-01 of historical
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     60225.0
    ...                        ...
    title:                     NorESM2-MM output prepared for CMIP6
    variable_id:               tslsi
    variant_label:             r1i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/e2022d36-165e-4824-b8f1-fe9dfe72625c

In [5]:
# Future Surface Temp 585
path="D:\Mosq Proj Data\CMIP Data\surface temperature\ssp585 CESM2 r11i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\surface temperature\ssp585 CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'tslsi'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Surface_Temp_Daily_Future_585=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Surface_Temp_Daily_Future_585['Future Surface Temp']=Data
Surface_Temp_Daily_Future_585['Date']=Date
Surface_Temp_Daily_Future_585['Future Surface Temp']=Surface_Temp_Daily_Future_585['Future Surface Temp']-273.15
Surface_Temp_Daily_Future_585['Count']=count

NameError: name 'zonal_stats' is not defined

In [ ]:
Surface_Temp_Daily_Future_585

In [ ]:
# Future Surface Temp 245
path="D:\Mosq Proj Data\CMIP Data\surface temperature\ssp245 CESM2"
basedir="D:\Mosq Proj Data\CMIP Data\surface temperature\ssp245 CESM2"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'tslsi'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area

            average[t] = np.average(mini, weights=f_overlap)
            
        else:
            average[t] = np.mean(mini)


Surface_Temp_Daily_Future_245=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Surface_Temp_Daily_Future_245['Future Surface Temp']=Data
Surface_Temp_Daily_Future_245['Date']=Date
Surface_Temp_Daily_Future_245['Future Surface Temp']=Surface_Temp_Daily_Future_245['Future Surface Temp']-273.15
Surface_Temp_Daily_Future_245['Count']=count

In [ ]:
Surface_Temp_Daily_Future_245

In [ ]:
# Historical Surface Temp
path="D:\Mosq Proj Data\CMIP Data\surface temperature\Hist CESM2 r11i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\surface temperature\Hist CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'tslsi'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area

            average[t] = np.average(mini, weights=f_overlap)
            

        else:
            average[t] = np.mean(mini)


Surface_Temp_Daily_Historical=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Surface_Temp_Daily_Historical['Historical Surface Temp']=Data
Surface_Temp_Daily_Historical['Date']=Date
Surface_Temp_Daily_Historical['Historical Surface Temp']=Surface_Temp_Daily_Historical['Historical Surface Temp']-273.15
Surface_Temp_Daily_Historical['Count']=count

In [ ]:
Surface_Temp_Daily_Historical

In [ ]:
# Future LAI 245
path="D:\Mosq Proj Data\CMIP Data\LAI\ssp245 CESM2 r11i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\LAI\ssp245 CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'lai'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


LAI_Daily_Future_245=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
LAI_Daily_Future_245['Future LAI']=Data
LAI_Daily_Future_245['Date']=Date
LAI_Daily_Future_245['Count']=count

In [ ]:
LAI_Daily_Future_245

In [ ]:
# Future LAI 585
path="D:\Mosq Proj Data\CMIP Data\LAI\ssp585 CESM2 r11i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\LAI\ssp585 CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'lai'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


LAI_Daily_Future_585=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
LAI_Daily_Future_585['Future LAI']=Data
LAI_Daily_Future_585['Date']=Date
LAI_Daily_Future_585['Count']=count

In [ ]:
LAI_Daily_Future_585

In [ ]:
# Historical LAI
path="D:\Mosq Proj Data\CMIP Data\LAI\Hist CESM2 r11i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\LAI\Hist CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'lai'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


LAI_Daily_Historical=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
LAI_Daily_Historical['Historical LAI']=Data
LAI_Daily_Historical['Date']=Date
LAI_Daily_Historical['Count']=count

In [ ]:
LAI_Daily_Historical

In [ ]:
# Future Precip 245
path="D:\Mosq Proj Data\CMIP Data\Precip\ssp245 CESM2"
basedir="D:\Mosq Proj Data\CMIP Data\Precip\ssp245 CESM2"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'pr'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Precip_Daily_Future_245=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Precip_Daily_Future_245['Future Precip']=Data
Precip_Daily_Future_245['Date']=Date
Precip_Daily_Future_245['Future Precip']=Precip_Daily_Future_245['Future Precip']*86400
Precip_Daily_Future_245['Count']=count

In [ ]:
# Future Precip 585
path="D:\Mosq Proj Data\CMIP Data\Precip\ssp585 CESM2 r11i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\Precip\ssp585 CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'pr'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Precip_Daily_Future_585=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Precip_Daily_Future_585['Future Precip']=Data
Precip_Daily_Future_585['Date']=Date
Precip_Daily_Future_585['Future Precip']=Precip_Daily_Future_585['Future Precip']*86400
Precip_Daily_Future_585['Count']=count

In [ ]:
# Historical Precip
path="D:\Mosq Proj Data\CMIP Data\Precip\Hist CESM2 r11i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\Precip\Hist CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'pr'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Precip_Daily_Historical=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Precip_Daily_Historical['Historical Precip']=Data
Precip_Daily_Historical['Date']=Date
Precip_Daily_Historical['Historical Precip']=Precip_Daily_Historical['Historical Precip']*86400
Precip_Daily_Historical['Count']=count

In [ ]:
Precip_Daily_Historical

In [ ]:
# Future Runoff 245
path="D:\Mosq Proj Data\CMIP Data\Runoff\ssp245 CESM2 r11i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\Runoff\ssp245 CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrro'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Runoff_Daily_Future_245=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Runoff_Daily_Future_245['Future Runoff']=Data
Runoff_Daily_Future_245['Date']=Date
Runoff_Daily_Future_245['Future Runoff']=Runoff_Daily_Future_245['Future Runoff']*(areaofshp[0])*0.001
Runoff_Daily_Future_245['Future_Runoff_Depth']=Runoff_Daily_Future_245['Future Runoff']*24*60*60/(areaofshp[0])
Runoff_Daily_Future_245['Count']=count

In [ ]:
# Future Runoff 585
path="D:\Mosq Proj Data\CMIP Data\Runoff\ssp585 CESM2 r11i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\Runoff\ssp585 CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrro'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.average(mini)


Runoff_Daily_Future_585=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Runoff_Daily_Future_585['Future Runoff']=Data
Runoff_Daily_Future_585['Date']=Date
Runoff_Daily_Future_585['Future Runoff']=Runoff_Daily_Future_585['Future Runoff']*(areaofshp[0])*0.001
Runoff_Daily_Future_585['Future_Runoff_Depth']=Runoff_Daily_Future_585['Future Runoff']*24*60*60/(areaofshp[0])
Runoff_Daily_Future_585['Count']=count

In [ ]:
# Historical Runoff
path="D:\Mosq Proj Data\CMIP Data\Runoff\Hist CESM2 r11i1p1f1"
basedir="D:\Mosq Proj Data\CMIP Data\Runoff\Hist CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrro'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Runoff_Daily_Historical=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Runoff_Daily_Historical['Historical Runoff']=Data
Runoff_Daily_Historical['Date']=Date
Runoff_Daily_Historical['Historical Runoff']=Runoff_Daily_Historical['Historical Runoff']*(areaofshp[0])*0.001
Runoff_Daily_Historical['Historical_Runoff_Depth']=Runoff_Daily_Historical['Historical Runoff']*24*60*60/(areaofshp[0])
Runoff_Daily_Historical['Count']=count

In [ ]:
Runoff_Daily_Historical

In [ ]:
# Future SM 245
path=r"D:\Mosq Proj Data\CMIP Data\upper sm content\ssp245 CESM2"
basedir=r"D:\Mosq Proj Data\CMIP Data\upper sm content\ssp245 CESM2"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrsos'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


SM_Daily_Future_245=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
SM_Daily_Future_245['Future SM']=Data
SM_Daily_Future_245['Date']=Date
SM_Daily_Future_245['Future SM']=SM_Daily_Future_245['Future SM']/100
SM_Daily_Future_245['Count']=count

In [ ]:
# Future SM 585
path=r"D:\Mosq Proj Data\CMIP Data\upper sm content\ssp585 CESM2 r11i1p1f1"
basedir=r"D:\Mosq Proj Data\CMIP Data\upper sm content\ssp585 CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrsos'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


SM_Daily_Future_585=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
SM_Daily_Future_585['Future SM']=Data
SM_Daily_Future_585['Date']=Date
SM_Daily_Future_585['Future SM']=SM_Daily_Future_585['Future SM']/100
SM_Daily_Future_585['Count']=count

In [ ]:
# Historical Soil Moisture
path=r"D:\Mosq Proj Data\CMIP Data\upper sm content\Hist CESM2 r11i1p1f1"
basedir=r"D:\Mosq Proj Data\CMIP Data\upper sm content\Hist CESM2 r11i1p1f1"
path_poly = r"D:\Mosquito Modeling\GTA boundaries\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]
count=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrsos'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


SM_Daily_Historical=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
SM_Daily_Historical['Historical SM']=Data
SM_Daily_Historical['Date']=Date
SM_Daily_Historical['Historical SM']=SM_Daily_Historical['Historical SM']/100
SM_Daily_Historical['Count']=count

In [ ]:
CMIP_Historical=pd.merge(LAI_Daily_Historical,Surface_Temp_Daily_Historical,how="left",on="Date")
CMIP_Historical=pd.merge(CMIP_Historical,Runoff_Daily_Historical,how="left",on="Date")
CMIP_Historical=pd.merge(CMIP_Historical,Precip_Daily_Historical,how="left",on="Date")
CMIP_Historical['Historical SM']=SM_Daily_Historical['Historical SM']

In [ ]:
CMIP_Future_245=pd.merge(LAI_Daily_Future_245,Surface_Temp_Daily_Future_245,how="left",on="Date")
CMIP_Future_245=pd.merge(CMIP_Future_245,Precip_Daily_Future_245,how="left",on="Date")
CMIP_Future_245=pd.merge(CMIP_Future_245,Runoff_Daily_Future_245,how="left",on="Date")
CMIP_Future_245['Future SM']=SM_Daily_Future_245['Future SM']

In [ ]:
CMIP_Future_585=pd.merge(LAI_Daily_Future_585,Surface_Temp_Daily_Future_585,how="left",on="Date")
CMIP_Future_585=pd.merge(CMIP_Future_585,Precip_Daily_Future_585,how="left",on="Date")
CMIP_Future_585=pd.merge(CMIP_Future_585,Runoff_Daily_Future_585,how="left",on="Date")
CMIP_Future_585['Future SM']=SM_Daily_Future_585['Future SM']

In [ ]:
CMIP_Future_245['Future Runoff'][CMIP_Future_245['Future Runoff']<0]=0
CMIP_Future_245['Future_Runoff_Depth'][CMIP_Future_245['Future_Runoff_Depth']<0]=0
CMIP_Future_245['Future Precip'][CMIP_Future_245['Future Precip']<0]=0

CMIP_Future_585['Future Runoff'][CMIP_Future_585['Future Runoff']<0]=0
CMIP_Future_585['Future_Runoff_Depth'][CMIP_Future_585['Future_Runoff_Depth']<0]=0
CMIP_Future_585['Future Precip'][CMIP_Future_585['Future Precip']<0]=0

CMIP_Historical['Historical Runoff'][CMIP_Historical['Historical Runoff']<0]=0
CMIP_Historical['Historical_Runoff_Depth'][CMIP_Historical['Historical_Runoff_Depth']<0]=0
CMIP_Historical['Historical Precip'][CMIP_Historical['Historical Precip']<0]=0

In [ ]:
CMIP_Historical=pd.merge(LAI_Daily_Historical,Surface_Temp_Daily_Historical,how="left",on="Date")
CMIP_Historical=pd.merge(CMIP_Historical,Runoff_Daily_Historical,how="left",on="Date")
CMIP_Historical=pd.merge(CMIP_Historical,Precip_Daily_Historical,how="left",on="Date")
CMIP_Historical['Historical SM']=SM_Daily_Historical['Historical SM']

CMIP_Historical['Month']=CMIP_Historical['Date']
CMIP_Historical['Day']=CMIP_Historical['Date']
CMIP_Historical['Year']=CMIP_Historical['Date']
for i in range(0,len(CMIP_Historical)):
    CMIP_Historical['Month'][i]=CMIP_Historical['Date'][i].month
    CMIP_Historical['Day'][i]=CMIP_Historical['Date'][i].day
    CMIP_Historical['Year'][i]=CMIP_Historical['Date'][i].year

In [ ]:
CMIP_Future_585

In [ ]:
CMIP_Future_245

In [ ]:
# CMIP_Future_245.to_excel('Path\To\File\CMIP_Future_CESM2_ssp245_Polygon.xlsx')
# CMIP_Future_585.to_excel('Path\To\File\CMIP_Future_CESM2_ssp585_Polygon.xlsx')
# CMIP_Historical.to_excel('Path\To\File\CMIP_Historical_CESM2_Polygon.xlsx')